In [1]:
import os
import glob
import random
import pathlib
import numpy as np
from tqdm import tqdm
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import DataLoader
import matplotlib.patches as patches
from PIL import Image
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
torch.backends.cudnn.benchmark = True

import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from matplotlib import animation
from matplotlib.animation import PillowWriter
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.colors as colors

import time
import multiprocessing as m
import concurrent.futures
from sklearn.metrics import confusion_matrix

import pickle

# Wave Propagation and Diffraction Constraints in a Photonic System

## 1. Wave Propagation in Photonics
Wave propagation in a photonic system refers to how light travels through a material, such as **silicon waveguides, photonic crystals, or free space**. The behavior of light in these systems is governed by **Maxwell's equations** and the material’s refractive index ($n$).

### Key properties influencing wave propagation:

#### Wavelength in material:
$
\lambda = \frac{\lambda_0}{n}
$
where $\lambda_0$ is the free-space wavelength, and $n$ is the refractive index.

#### Frequency of light:
$
f = \frac{c}{\lambda}
$
where $c$ is the speed of light.

---

## 2. Diffraction Constraints in Photonic Systems
Diffraction occurs when light waves **bend or spread** after passing through an opening or interacting with an obstacle. In photonics, diffraction constraints arise due to the **spatial resolution limits** of a structure, such as in **photonic integrated circuits (PICs) and nanophotonics**.

### Key diffraction constraints:

#### Minimum feature size (resolution limit):
$
d_{\text{min}} \geq \frac{\lambda}{2n}
$
This defines the smallest pattern that can be resolved in a photonic circuit.

#### Diffraction-limited propagation distance (Rayleigh length $z_R$):
$
z_R = \frac{\pi w_0^2}{\lambda}
$
where $w_0$ is the beam waist.

#### Evanescent coupling:
Light can leak into adjacent structures if spacing is **below a critical threshold**, which depends on the refractive index contrast.

---

## 3. Application in Code
The formula in code:
$
z_{\text{min}} = \text{size} \times \text{rect\_length} \times \sqrt{\frac{4 \times \text{rect\_length}^2}{wv^2} - 1} \times 10^6
$


1. **The photonic structure must be large enough** to ensure wave connectivity
2. **Diffraction effects are considered**  since the term inside the square root resembles a diffraction condition
3. **The structures minimum distance is set to prevent optical signal loss** or unwanted interference

---


In [2]:
resample_size = 10 # down sample from 28 to 10
thold = 0.22
pad_size = 0
size = resample_size*resample_size + 2*pad_size

ref_ind = 3.45          # Refractive index of Silicon
free_wv = 1.55e-6       # Wavelength in freespace
wv = free_wv/ref_ind    # Wavelength in Silicon

Hz = 3e8/wv
rect_length = 0.5e-6
layer_length = size*rect_length

z_min = size * rect_length * np.sqrt((4*rect_length**2)/wv**2-1) *1e6
print("The minimum distance to ensure full connectivity is: " + str(z_min))
z = 100e-6 # 300e-6

num_layers = 1
lr = 0.1
batch_size = 100
epochs = 200

# RAM 64GB,8cpu ,size =100, num_process = 20, W --> 63 sec
# RAM 64GB,8cpu ,size =120, num_process = 20, W --> 129 sec

The minimum distance to ensure full connectivity is: 99.42603230595124


# Build and train two-dimensional Diffractive Deep Neural Network

A D2NN is a type of deep neural network that processes optical signals through diffraction instead of traditional electronic computations

In [3]:
def core_matrix(z):
    distance = z                         # distance bewteen two layers (3cm)
    wl = 3e8 / Hz                        # wave length
    wn = 2 * np.pi / wl                  # wave number
    c = rect_length
    w = np.zeros((size, size), dtype=np.complex64)

    for i in range(0, size, 1):
        s = i % size
        for k in range(i+1, size, 1):
            n = k % size
            # print(n-s)
            r_ns = np.sqrt(np.square(distance) + np.square(n * c - s * c))
            w[i][k] = np.exp(1j * wn * r_ns) * (1 / (2 * np.pi * r_ns) + 1 / (1j * wl)) * distance / (r_ns ** 2)
            

    w_diagonal = np.exp(1j * wn * distance) * (1 / (2 * np.pi * distance) + 1 / (1j * wl)) * distance / (distance*distance)
    w = w.T + w

    for i in range(0, size, 1):
        w[i][i] = w_diagonal
    
    w = (w * rect_length * 0.22e-6)     # !!!!!!!!!!!!
    return w

start = time.time()
W = core_matrix(z)
end = time.time()
print('NumPy computation time：', end - start, 's')

print(np.shape(W))
# np.savetxt(r'W.txt', np.array(W),delimiter = ',')

NumPy computation time： 0.04548048973083496 s
(100, 100)


In [4]:
def convert_to_col_vector(arr):
    vec = arr.view(arr.size(0), -1).t()
    return vec

In [7]:
def convert_to_mat(arr):
    mat = arr.t().view(arr.size(1), size, 1)
    return mat

In [9]:
def detector_region(x):  
    '''extract and normalize the output intensity distribution'''
    i = size/165
    m = 10
    step_sizes = [10, 25, 40, 55, 70, 85, 100, 115, 130, 145]
    result = []
    for step in step_sizes:
        slice = x[:, int(i*step):int(i*(step+m)), :]
        result.append((slice / torch.max(slice)).mean(dim=(1, 2)).unsqueeze(-1))
    return torch.cat(result, dim=-1)